In [ ]:
import junodch_utils_read_img as utils

import matplotlib.pyplot as matPlt
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.keras import TqdmCallback

import keras
from keras import layers, losses
from keras.models import Model
from shapely.geometry import box
#import gdal

import rasterio
from rasterio import plot as rastPlt
from rasterio.merge import merge as rasterMerge

from ipyleaflet import *
from localtileserver import get_leaflet_tile_layer, TileClient
from geemap import *

In [ ]:
autoencoder = keras.models.load_model('model/autoencoder_64_V2')

In [ ]:
folderName = "img/Sokoto/"
satelliteFile = "Sentinel-2_1.tif"
#nightFile = "Night VIIRS_1.tif"
nightFile = "lowres_night_1.tif"

# Filter the night tile not covering the day area.
with rasterio.open(folderName + satelliteFile) as s:
  sBox = box(*s.bounds).exterior.coords
aoi = []
for v in sBox:
  aoi.append((v[0], v[1]))
aoi.pop()

coords, test = utils.getTrainingAndTestPerimeter(folderName + nightFile, 0, area=aoi)

print("TrainingTile:",len(coords))
print("TestTile:",len(test))


In [ ]:
#with rasterio.open(folderName + satelliteFile) as s:
#  validation, metaValid = utils.getEachImgFromCoord(s, coords[2944:2954], True)

#data = utils.formatDataForAutoencoder(validation)

#utils.displayAutoEncoderResults(data, None, autoencoder, keras.losses.MeanAbsoluteError())

In [ ]:
#cVal = 736 # TODO to delete

batch = 256
iSave = 0
iNext = 0

#coordsTest = coords[2208:2308] + coords[2944:3044] + coords[3680:3780]
coordsTest = coords

size = len(coordsTest)

lossFunction = keras.losses.MeanSquaredError()
result = [0]*len(coordsTest)

with rasterio.open(folderName + satelliteFile) as s:
  while iNext < size:
    iNext = iSave+batch
    if iNext > size: 
      iNext = size
    dataInput, metaValid = utils.getEachImgFromCoord(s, coordsTest[iSave:iNext], True)
    dataformated = utils.formatDataForAutoencoder(dataInput,res=64)
    decoded_imgs = autoencoder.predict(dataformated,verbose=0,)
    for i in range(iNext-iSave):
      result[iSave+i] = np.round(lossFunction(dataformated[i], decoded_imgs[i]).numpy(),0)
    iSave = iNext
  


In [ ]:
batch = 256
iSave = 0
iNext = 0
size = len(coordsTest)
resultValid = [False]*len(coordsTest)

with rasterio.open(folderName + 'Population GHSL_1.tif') as p:
  while iNext < size:
    iNext = iSave+batch
    if iNext > size: 
      iNext = size
    validationData, _ = utils.getEachImgFromCoord(p, coordsTest[iSave:iNext], True)
    for i in range(iNext-iSave):
      resultValid[iSave+i] = any(any(c > 200 for c in row) for row in validationData[i][0])
    iSave = iNext

In [ ]:
with rasterio.open(folderName + nightFile) as s:
  testData, testMeta = utils.getEachImgFromCoord(s, coordsTest, True)

def setVal2x2(val):
  return [[val,val],[val,val]]

def setThreshold(val, isInPop, minExpected, maxExpected):
  rgb = (0,0,0)
  if val > maxExpected: # unkown data
    rgb = (255,255,0) if isInPop else(0,0,255)
  elif val >= minExpected and val <= maxExpected: # Settlement detected
    rgb = (0,255,0) if isInPop else (255,0,0)
  elif val < minExpected: # Empty
    rgb = (100,100,100) if isInPop else (0,0,0)
  return [setVal2x2(rgb[0]), setVal2x2(rgb[1]), setVal2x2(rgb[2]), setVal2x2(255)]


resultImg = [setThreshold(result[i], resultValid[i], 100, 300) for i in range(len(result))]
for i, m in enumerate(testMeta):
  testMeta[i] = rasterio.Affine(m[0]/2, m[1], m[2], m[3], m[4]/2, m[5])

In [ ]:
#utils.displayTiles(dataInput, metaValid)
#with rasterio.open(folderName + "Sentinel-2.tif") as s:
  #utils.displayTiles([s.read()], [s.transform])
  #utils.displayTiles([s.read()] + resultImg, [s.transform] + testMeta)

fig, axs = matPlt.subplots(1,3, dpi=240)
with rasterio.open(folderName + satelliteFile) as s:
  utils.displayTiles([s.read()], [s.transform],axs[0])

axs[2].set_xlim(axs[0].get_xlim())
axs[2].set_ylim(axs[0].get_ylim())
with rasterio.open(folderName + 'Population GHSL_1.tif') as p:
  rastPlt.show(p, ax=axs[2])

utils.displayTiles(resultImg, testMeta, axs[1])


In [ ]:
m = Map(center=(0, 0), zoom=4)

display(m)